sync linkedin contacts with google contacts?



In [ ]:
var importer = require('../Core');
var fs = require('fs');
var glob = require('glob');
var path = require('path');
var {
    updateContact,
    runSeleniumCell
} = importer.import([
    'update google contact',
    'selenium cell'
]);
var listAllConnections, scrapeEntireLinkedInProfile;

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';
function escapeFilename(f) {
    return f.replace(/[^a-z0-9-\s]/ig, '_')
}

function syncLinkedInContacts() {
    var loaded = glob.sync('**/linkedin-*.json', {cwd: project})
        .reduce((arr, l) => {
            try {
                arr.push(escapeFilename(path.basename(l).replace('.json', '').replace('linkedin-', '')))
                arr.push(escapeFilename(JSON.parse(fs.readFileSync(path.join(project, l)))[0]
                    .url.replace(/^.*?\/*in\/|\/$/ig, '')));
            } catch (e) {
                console.log(l);
            }
            return arr;
        }, []);
    return runSeleniumCell([
        'log in linkedin',
        'scrape linkedin profile',
        'scrape linkedin contacts',
        'scrape entire linkedin profile'
    ])
        .then(r => ({
            listAllConnections, scrapeEntireLinkedInProfile
        } = r).loginLinkedIn())
        .then(() => listAllConnections())
        .then((connections) => {
            const filtered = connections.filter(c => {
                const fname = escapeFilename(c.replace(/^.*?\/*in\/|\/$/ig, ''));
                return loaded.indexOf(fname) === -1
                    && loaded.indexOf(c) === -1
            });
            const percent = Math
                .round((connections.length - filtered.length) / connections.length * 100);
            console.log((connections.length - filtered.length)
                + ' / ' + connections.length + ' : '
                + percent
                + '%');
            console.log(filtered.slice(0, 30));
            return importer.runAllPromises(filtered.slice(0, 30).map(c => resolve => {
                var linkedIn;
                const fname = c.replace(/^\/*in\/|\/$/ig, '');
                return scrapeEntireLinkedInProfile('https://linkedin.com' + c)
                    .then(r => {
                        linkedIn = r;
                        return updateContact({
                            displayName: r.name.split(/\s/).join('.*')
                        });
                    })
                    .then(r => {
                        console.log(linkedIn.name);
                        if (r.length === 0) {
                            r.push({});
                        }
                        Object.assign(r[0], linkedIn);
                        fs.writeFileSync(
                            project + '/linkedin-' + escapeFilename(fname) + '.json',
                            JSON.stringify(r, null, 4));
                        return resolve(r);
                    })
                    .catch(e => {
                        console.log(e);
                        resolve()
                    })
            }))
        })
        .catch(e => console.log(e))
};

module.exports = syncLinkedInContacts;



scrape entire linkedin profile?



In [ ]:
var importer = require('../Core');

function loadEntirePage() {
    return client
    // load all defered elements by scrolling
        .then(() => getAllUntil(
            false,
            '//*[contains(@class, "pv-deferred-area")]/@id',
            [],
            (a, b) => a === b,
            (i) => i < 30
        ))
        .pause(2000)
        // show more all sections
        .scroll(0, 0)
        .then(() => importer.runAllPromises([
            '//button[contains(., "See more")]',
            '//button[contains(., "more skills")]',
            '//button[contains(., "more education")]',
            '//button[contains(., "more recommendations")]'
        ].map(selector => resolve => {
            return client
                .isVisible(selector)
                .then(is => is
                    ? client.click(selector).pause(1000)
                    : [])
                .then(e => resolve())
                .catch(e => {console.log; return resolve()})
        })))
        .catch(e => console.log(e))
};

function scrapeEntireLinkedInProfile(profile) {
    var contact;
    return client
        .then(() => clickSpa(profile))
        .then(() => readLinkedInProfileInfo())
        .then(r => {
            contact = r;
            return loadEntirePage();
        })
        // get extra profile info
        .then(() => {
            return getAllXPath({
                summary: '//p[contains(@class, "section__summary")]//text()',
                experience: [
                    '//*[contains(@class, "profile-section__card")]|//*[contains(@class, "profile-section__section-info-item")]',
                    ['.//text()']
                ],
                skills: [
                    '//*[contains(@class, "skill-entity--featured")]|//*[contains(@class, "skill-entity__pill")]',
                    ['.//text()']
                ],
                recommendations: [
                    '//*[contains(@class, "recommendation-entity")]',
                    ['.//text()']
                ],
                interests: [
                    '//*[contains(@class, "interest-entity")]',
                    ['.//text()']
                ]
            });
        })
        .then(r => {
            return Object.assign(r, {
                summary: r.summary.join(''),
                experience: r.experience.map(e => typeof e[0] === 'string'
                    ? e[0]
                    : e[0].join('')
                        .trim().split(/\s*\n+\s*/)),
                skills: r.skills.map(e => typeof e[0] === 'string'
                    ? e[0]
                    : e[0].join('')
                        .trim().split(/\s*\n+\s*/)),
                recommendations: r.recommendations.map(e => typeof e[0] === 'string'
                    ? e[0]
                    : e[0].join('')
                        .trim().split(/\s*\n+\s*/)),
                interests: r.interests.map(e => typeof e[0] === 'string'
                    ? e[0]
                    : e[0].join('')
                        .trim().split(/\s*\n+\s*/))
            }, contact)
        })
        .catch(e => console.log(e))
};
module.exports = scrapeEntireLinkedInProfile;



scrape linkedin contacts?



In [ ]:
var importer = require('../Core');
var fs = require('fs');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

function listAllConnections() {
    var friends = [];
    if (fs.existsSync(project + '/connections.json')) {
        return Promise.resolve(JSON.parse(
            fs.readFileSync(project + '/connections.json')));
    }
    return client
        .getUrl()
        .url('https://www.linkedin.com/mynetwork/invite-connect/connections/')
        .pause(3000)
        .then(() => getAllUntil(
            false,
            '//a[contains(@href, "/in/")]/@href',
            friends,
            (a, b) => a === b,
            (i) => i < 100
        ))
        .then(r => r.filter((l, i, arr) => arr.indexOf(l) === i))
        .then(r => {
            fs.writeFileSync(
                project + '/connections.json',
                JSON.stringify(r, null, 4));
            return r;
        })
        .catch(e => console.log(e))
};
module.exports = listAllConnections;


test linkedin scraper


In [ ]:
var importer = require('../Core');
var syncLinkedInContacts = importer.import('sync linkedin contacts');
$$.async();
syncLinkedInContacts()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));



Initializing webdriver on localhost
1413 / 2435 : 58%
[ '/in/gail-barnes-pmp%C2%AE-csm-mba-2987a11/',
  '/in/kimberly-rizzo-63456971/',
  '/in/peter-piotr-g%C3%B3ralczyk-goralczyk-940a5222/',
  '/in/derek-stewart-2a2074103/',
  '/in/christian-h%C3%A9bert-ba27b8125/',
  '/in/%F0%9F%8C%9Fjonathan-barry%F0%9F%8C%9F-92aa8016/',
  '/in/michaeldpeters/',
  '/in/chris-huie-896936b2/',
  '/in/david-cohen-7375728/',
  '/in/joshua-bamidele-%C2%AE9k-connections%E2%84%A2-3a1214b4/',
  '/in/chalon-green-3b0b6a2/',
  '/in/kennethervinyoung/',
  '/in/vgdgpforza/',
  '/in/katelynedmonds/',
  '/in/themikstateam/',
  '/in/alan-masters-919579100/',
  '/in/michael-fitzgibbon-757aaa30/',
  '/in/neilticktin/',
  '/in/tony-ardizzone-b108878/',
  '/in/marissa-piasecki-0a06224a/',
  '/in/gailconn/',
  '/in/lauren-brooke-miller-59a8602a/',
  '/in/christopheragraham/',
  '/in/ronenmagid/',
  '/in/jasonet/',
  '/in/melissaerinmcfarlin/',
  '/in/anastasiagamick/',
  '/in/markbarrenechea/',
  '/in/daniellesharonabr

connect on linkedin?


In [8]:
var importer = require('../Core');
var fs = require('fs');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

function listAllConnections() {
    var friends = [];
    if (fs.existsSync(project + '/new-connections.json')) {
        return Promise.resolve(JSON.parse(
            fs.readFileSync(project + '/new-connections.json')));
    }
    return client
        .getUrl()
        .url('https://www.linkedin.com/mynetwork/')
        .pause(3000)
        .then(() => getAllUntil(
            false,
            '//a[contains(@href, "/in/")]/@href',
            friends,
            (a, b) => a === b,
            (i) => i < 100
        ))
        .then(r => r.filter((l, i, arr) => arr.indexOf(l) === i))
        .then(r => {
            fs.writeFileSync(
                project + '/new-connections.json',
                JSON.stringify(r, null, 4));
            return r;
        })
        .catch(e => console.log(e))
};

function connectLinkedin(profile) {
    return client
        .clickSpa('https://linkedin.com' + profile)
        .isExisting('//button[contains(., "Connect")]')
        .then(is => is
              ? client
              .isVisible('//button[contains(., "Connect")]')
              .then(is => is
                    ? client
                    .click('//button[contains(., "Connect")]')
                    .pause(1000)
                    : [])
              : [])
        .isExisting('//button[contains(., "Send now")]')
        .then(is => is
              ? client
              .isVisible('//button[contains(., "Send now")]')
              .then(is => is
                    ? client
                    .click('//button[contains(., "Send now")]')
                    .pause(1000)
                    : [])
              : [])
        .catch(e => console.log(e))
}

module.exports = {
    listAllConnections,
    connectLinkedin
};


{ listAllConnections: [Function: listAllConnections],
  connectLinkedin: [Function: connectLinkedin] }

connect add friends linkedin?


In [1]:
var importer = require('../Core');
var glob = require('glob');
var path = require('path');
var fs = require('fs');
var runSeleniumCell = importer.import('run selenium cell');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';


var listAllConnections, connectLinkedin, loginLinkedIn;
function addLinkedinConnections() {
    var pages;
    return runSeleniumCell(['log in linkedin', 'connect on linkedin'])
        .then(r => {
            listAllConnections = r[1].listAllConnections;
            connectLinkedin = r[1].connectLinkedin;
            loginLinkedIn = r.loginLinkedIn;
            return loginLinkedIn();
        })
        .then(() => listAllConnections())
        .then(pages => importer.runAllPromises(pages.slice(0, 30).map(page => resolve => {
            console.log(page);
            return connectLinkedin(page, true)
                .then(r => resolve(r))
                .catch(e => resolve(e))
        })))
        .catch(e => console.log(e))
}
module.exports = addLinkedinConnections;



[Function: addLinkedinConnections]

In [2]:
$$.async();
addLinkedinConnections()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))


Initializing webdriver on localhost
[ 1511149306836, 'ec2eca85b6fa3331aa4d685bb091fd7b' ]
[20:54:09]  COMMAND	GET 	 "/wd/hub/session/ec2eca85b6fa3331aa4d685bb091fd7b/window_handle"
[20:54:09]  COMMAND	POST 	 "/wd/hub/session/ec2eca85b6fa3331aa4d685bb091fd7b/window"
[20:54:09]  COMMAND	GET 	 "/wd/hub/session/ec2eca85b6fa3331aa4d685bb091fd7b"
[ 'ec2eca85b6fa3331aa4d685bb091fd7b' ]
[20:54:09]  COMMAND	GET 	 "/wd/hub/session/ec2eca85b6fa3331aa4d685bb091fd7b"
ec2eca85b6fa3331aa4d685bb091fd7b
[20:54:09]  COMMAND	GET 	 "/wd/hub/session/ec2eca85b6fa3331aa4d685bb091fd7b/window_handle"
[20:54:09]  COMMAND	POST 	 "/wd/hub/session/ec2eca85b6fa3331aa4d685bb091fd7b/window"
[20:54:09]  COMMAND	GET 	 "/wd/hub/session/ec2eca85b6fa3331aa4d685bb091fd7b"
[20:54:09]  COMMAND	POST 	 "/wd/hub/session/ec2eca85b6fa3331aa4d685bb091fd7b/execute"
[20:54:09]  COMMAND	GET 	 "/wd/hub/session/ec2eca85b6fa3331aa4d685bb091fd7b/window_handles"
[20:54:09]  COMMAND	POST 	 "/wd/hub/session/ec2eca85b6fa3331aa4d685bb091fd7b/

[]